### Archived Statements - part 4

Target:

 - us-east = 809,993,275
 - us-west = 809,993,275
 - archives = 3,239,979,516 + 2,159,986,344 + 2,159,986,344= 8,639,945,376

 
Process:

 - bulk import via stream consumer of import/statements_archive4
 - use externalScroller to bulk index at the same time

#### Run the imports

In [ ]:
!import.sh -o import -t 24 -l import/statements_archive4 -r archives -z 500 -b / -a -w 200000 -bulk -useScroller > logs/import-archives-step4.log

Started BulkIndex using ExternalScroller: aa27c114-50a1-452f-bb67-2f02df3381d9

In [ ]:
# check status
! cid="aa27c114-50a1-452f-bb67-2f02df3381d9"; \
  INJECTOR="http://127.0.0.1:8080"; \
  curl -H 'Content-Type:application/json+nxrequest' -H 'X-NXRepository:archives' \
  -u $NXUSER:$NXPWD $INJECTOR/nuxeo/api/v1/bulk/$cid